In [1]:
import pandas as pd
import scipy.sparse as sparse
import numpy as np
from scipy.sparse.linalg import spsolve

In [2]:
grouped_purchased = pd.read_csv('cvs_data_file/online-retail.csv', header = 0)
grouped_purchased.head()

,CustomerID,StockCode,Quantity
0,12346,23166,1
1,12347,16008,24
2,12347,17021,36
3,12347,20665,6
4,12347,20719,40


In [3]:
customers = list(np.sort(grouped_purchased.CustomerID.unique())) # Get our unique customers
products = list(grouped_purchased.StockCode.unique()) # Get our unique products that were purchased
quantity = list(grouped_purchased.Quantity) # All of our purchases

rows = grouped_purchased.CustomerID.astype('category', categories = customers).cat.codes 
# Get the associated row indices
cols = grouped_purchased.StockCode.astype('category', categories = products).cat.codes 
# Get the associated column indices
purchases_sparse = sparse.csr_matrix((quantity, (rows, cols)), shape=(len(customers), len(products)))

C:\Users\Office\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  """
C:\Users\Office\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  import sys


In [4]:
matrix_size = purchases_sparse.shape[0]*purchases_sparse.shape[1] # Number of possible interactions in the matrix
num_purchases = len(purchases_sparse.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

98.32190920694744

In [5]:
import random

In [6]:
def make_train(ratings, pct_test = 0.2):
    '''
    This function will take in the original user-item matrix and "mask" a percentage of the original ratings where a
    user-item interaction has taken place for use as a test set. The test set will contain all of the original ratings, 
    while the training set replaces the specified percentage of them with a zero in the original ratings matrix. 
    
    parameters: 
    
    ratings - the original ratings matrix from which you want to generate a train/test set. Test is just a complete
    copy of the original set. This is in the form of a sparse csr_matrix. 
    
    pct_test - The percentage of user-item interactions where an interaction took place that you want to mask in the 
    training set for later comparison to the test set, which contains all of the original ratings. 
    
    returns:
    
    training_set - The altered version of the original data with a certain percentage of the user-item pairs 
    that originally had interaction set back to zero.
    
    test_set - A copy of the original ratings matrix, unaltered, so it can be used to see how the rank order 
    compares with the actual interactions.
    
    user_inds - From the randomly selected user-item indices, which user rows were altered in the training data.
    This will be necessary later when evaluating the performance via AUC.
    '''
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of user,item index into list
    random.seed(0) # Set the random seed to zero for reproducibility
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of user-item pairs without replacement
    user_inds = [index[0] for index in samples] # Get the user row indices
    item_inds = [index[1] for index in samples] # Get the item column indices
    training_set[user_inds, item_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    return training_set, test_set, list(set(user_inds)) # Output the unique list of user rows that were altered  

In [7]:
product_train, product_test, product_users_altered = make_train(purchases_sparse, pct_test = 0.2)

In [8]:
item_lookup = pd.read_csv('cvs_data_file/item_lookup.csv', header = 0)
item_lookup.head()

,StockCode,Description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [9]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k,auc_score


# Instantiate and train the model
model = LightFM(loss='warp')
model.fit(product_train, epochs=40, num_threads=2)

# Evaluate the trained model
test_precision = precision_at_k(model,product_test, k=5).mean()
train_precision = precision_at_k(model, product_train, k=10).mean()

train_auc = auc_score(model, product_train).mean()
test_auc = auc_score(model, product_test).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))


C:\Users\Office\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


Precision: train 0.28, test 0.36.
AUC: train 0.92, test 0.91.


In [11]:
def display_recommended_items(model, data, user_ids):
    
    n_users, n_items = data.shape

    for id in user_ids:

        known_positives = item_lookup['Description'][product_train.tocsr()[id].indices]
        known_positives_df = pd.DataFrame(data=known_positives)
        print(known_positives_df,'\n','######################################')            
       
        scores = model.predict(id, np.arange(n_items))

        top_items = item_lookup['Description'][np.argsort(-scores)]
        df = pd.DataFrame(data=top_items)
        print(df)            
            

display_recommended_items(model, product_train, [34])


                              Description
72            60 TEATIME FAIRY CAKE CASES
154          CHRISTMAS LIGHTS 10 REINDEER
155      VINTAGE UNION JACK CUSHION COVER
157                RED RETROSPOT UMBRELLA
169       DISCO BALL CHRISTMAS DECORATION
170                  SMALL POPCORN HOLDER
171                 LARGE POPCORN HOLDER 
172   SET/20 RED RETROSPOT PAPER NAPKINS 
242                   BATHROOM METAL SIGN
243                    KITCHEN METAL SIGN
244                     TOILET METAL SIGN
330               CHILDREN'S SPACEBOY MUG
331                 FELTCRAFT CUSHION OWL
518            CINAMMON SET OF 9 T-LIGHTS
519         ORANGE SCENTED SET/9 T-LIGHTS
520           RIBBON REEL STRIPES DESIGN 
579                     HERB MARKER THYME
641            3 HOOK HANGER MAGIC GARDEN
692                 MINI JIGSAW DINOSAUR 
777     SET OF SALT AND PEPPER TOADSTOOLS
778          HOOK, 1 HANGER ,MAGIC GARDEN
789            SKULLS AND CROSSBONES WRAP
830         SET OF 4 FAIRY CAKE PL